In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
sys.path.append('..\\scripts\\')

In [3]:
from Malla_Validacion import validacion

In [4]:
EmprendimientoColectivo = validacion(nombre_api = 'token_EmprendimientoColectivo', json_malla = False, nombre_malla = 'Malla_Emprendimiento_Colectivo')

In [5]:
EmprendimientoColectivo.get_dataframe()
EmprendimientoColectivo.normalize_data()

Dataframe Cargado desde el API


In [6]:
data = EmprendimientoColectivo.dataframe

In [24]:
data.head()

,id,idencuesta,estado,identificacion,descripcion,fechaenviado,usuarioId,NUMERODOCUMENTOTITULAR,NOMBRECOMPLETO,CODDPTOATENCION,...,vlr_ingr_Cosecha,EdadExped,Calcaños,num_sem_buscando,ind_ingr_estado.ind_ingr_estado.¿Cuánto recibió?,ind_ingr_remesa_pais.ind_ingr_remesa_pais.¿Cuánto recibió?,ind_ingr_arriendos.ind_ingr_arriendos.¿Cuánto recibió?,ind_ingr_remesa_exterior.ind_ingr_remesa_exterior.¿Cuánto recibió?,ind_ingr_pension.ind_ingr_pension.¿Cuánto recibió?,ind_otros_ingresos.ind_otros_ingresos.¿Cuánto recibió?
0,582507,212,1,3640739,RAFAEL ANGEL MURILLO ALVAREZ,03/11/2023 18:05:20,2246,3640739,RAFAEL ANGEL MURILLO ALVAREZ,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,582509,212,1,3969671,UBALDO MEDRANO BERRIO,02/11/2023 15:05:34,2248,3969671,UBALDO MEDRANO BERRIO,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,582514,212,1,15241727,REYES STEPHENS BENT,02/11/2023 21:26:16,2248,15241727,REYES STEPHENS BENT,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,582520,212,1,18002086,DANA DALMIRO PRYME PUSEY,03/11/2023 23:24:46,2247,18002086,DANA DALMIRO PRYME PUSEY,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,582522,212,1,18003016,FERNANDO ANTONIO HOOKER ALVARADO,02/11/2023 23:38:47,2245,18003016,FERNANDO ANTONIO HOOKER ALVARADO,88,...,12000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
producto_servicio = pd.json_normalize(data['producto/servicio'].explode('producto/servicio'))
relacion_filas = data['producto/servicio'].explode().index
producto_servicio['id'] = relacion_filas

In [16]:
test = producto_servicio.groupby('id', as_index=False).agg({'BIENES_SERVICIOS':lambda x: np.nan if all(pd.isna(m) for m in x) else [i for i in x],
                                                     'PROMEDIO_VOLUMEN_MENSUAL_CANTIDAD': lambda x: np.nan if all(pd.isna(m) for m in x) else [i for i in x],
                                                     'PROMEDIO_VOLUMEN_MENSUAL_UNIDAD_MEDIDA': lambda x: np.nan if all(pd.isna(m) for m in x) else [i for i in x],
                                                     'ALMACENAR_PROVISIONES':lambda x: np.nan if all(pd.isna(m) for m in x) else [i for i in x],
                                                     'PROCESOS_AGREGACION_VALOR_PRODUCTOS': lambda x: [i for i in x],
                                                     'VALOR_PROMEDIO_MENSUAL_BIENES_SERVICIOS': lambda x: np.nan if all(pd.isna(m) for m in x) else [i for i in x]})

In [17]:
def remove_nans(row, col):
    A = row[col]
    valor = pd.isna(A).squeeze()
    forma = len(valor.shape)
    if forma == 0:
        if valor == True:
            return np.nan
        else:
            return A
    elif forma == 1:
        if all(i for i in valor):
            return np.nan
        else:
            return A
    else:
        if all(all(m for m in i) for i in valor):
            return np.nan
        else:
            return A

In [18]:
test['PROCESOS_AGREGACION_VALOR_PRODUCTOS'] = test.apply(remove_nans, args=('PROCESOS_AGREGACION_VALOR_PRODUCTOS',), axis= 1)

In [8]:
EmprendimientoColectivo.get_malla()

In [9]:
malla = EmprendimientoColectivo.malla

In [27]:
is_dict = data.transform(lambda x: x.apply(type).eq(list)).any()
posible_expandir = list(is_dict[is_dict==True].index)
Expandir = [i for i in posible_expandir if malla[i]['valores'] is None]

In [28]:
producto_servicio = pd.json_normalize(data['producto/servicio'].explode())
relacion_filas = data['producto/servicio'].explode().index
producto_servicio['id'] = relacion_filas

In [30]:
is_dict = producto_servicio.transform(lambda x: x.apply(type).eq(list)).any()
a_expandir = list(is_dict[is_dict==True].index)

In [44]:
a_expandir
columnas = list(producto_servicio.columns)
columnas.remove("id")
columnas

['BIENES_SERVICIOS',
 'PROMEDIO_VOLUMEN_MENSUAL_CANTIDAD',
 'PROMEDIO_VOLUMEN_MENSUAL_UNIDAD_MEDIDA',
 'ALMACENAR_PROVISIONES',
 'PROCESOS_AGREGACION_VALOR_PRODUCTOS',
 'VALOR_PROMEDIO_MENSUAL_BIENES_SERVICIOS']

In [53]:
producto_servicio.groupby('id', as_index=False).agg(aggregation_dict)

,id,BIENES_SERVICIOS,PROMEDIO_VOLUMEN_MENSUAL_CANTIDAD,PROMEDIO_VOLUMEN_MENSUAL_UNIDAD_MEDIDA,ALMACENAR_PROVISIONES,PROCESOS_AGREGACION_VALOR_PRODUCTOS,VALOR_PROMEDIO_MENSUAL_BIENES_SERVICIOS
0,0,NaN,NaN,NaN,NaN,[nan],NaN
1,1,NaN,NaN,NaN,NaN,[nan],NaN
2,2,[Pesca artesanal],[1200000.0],[Kilos],"[En un local, vivienda, finca o terreno propio]",[[Almacenamiento]],[ Más de $900.000 pesos.]
3,3,NaN,NaN,NaN,NaN,[nan],NaN
4,4,NaN,NaN,NaN,NaN,[nan],NaN
...,...,...,...,...,...,...,...
196,196,"[Uñas semipermanentes, Uñas basicas]","[90000.0, 4.0]","[Unidades, Litros]","[No tienen ningún lugar de acopio, No tienen n...","[nan, nan]","[ Entre $100.000 y menos de $200.000 pesos., ..."
197,197,[Peritaje forence psicologico],[3.0],[Litros],"[En un local, vivienda, finca o terreno en arr...",[nan],[ Entre $800.000 y menos de $900.000 pesos.]
198,198,"[Calculo de red y regulacion, Fabriccion e ins...","[4.0, 1.0, 8.0, 5.0]","[Unidades, Unidades, Unidades, Unidades]","[No tienen ningún lugar de acopio, No tienen n...","[[Transformación], [Transformación], [Transfor...","[ Entre $700.000 y menos de $800.000 pesos., ..."
199,199,NaN,NaN,NaN,NaN,[nan],NaN


In [52]:
aggregation_dict = {
    columna: (lambda x: [i for i in x]) if columna in a_expandir else (lambda x: [i for i in x] if not all(pd.isna(i) for i in x) else np.nan)
    for columna in columnas
}

In [54]:
def remove_nans(row, col):
    A = row[col]
    valor = pd.isna(A).squeeze()
    forma = len(valor.shape)
    if forma == 0:
        if valor == True:
            return np.nan
        else:
            return A
    elif forma == 1:
        if all(i for i in valor):
            return np.nan
        else:
            return A
    else:
        if all(all(m for m in i) for i in valor):
            return np.nan
        else:
            return A

In [84]:
def expand_data_frame(col, data):
    expansion = pd.json_normalize(data[col].explode())
    relacion_filas = data[col].explode().index
    
    if len(list(expansion.columns)) == 1:
        expansion = expansion.rename(columns={list(expansion.columns)[0]:col})
        
    expansion['id'] = relacion_filas
    
    
    is_dict = expansion.transform(lambda x: x.apply(type).eq(list)).any()
    a_expandir = list(is_dict[is_dict==True].index)
    
    columnas = list(expansion.columns)
    columnas.remove("id")
    
    aggregation_dict = {
    columna: (lambda x: [i for i in x]) if columna in a_expandir else (lambda x: [i for i in x] if not all(pd.isna(i) for i in x) else np.nan)
    for columna in columnas
    }
    
    result = expansion.groupby('id', as_index=False).agg(aggregation_dict)
    
    for columna in a_expandir:
        result[columna] = result.apply(remove_nans, args=(columna,), axis = 1)
    
    result = result.drop(columns='id')
    return result
    

In [85]:
def expand_cols(data:pd.DataFrame, diccionario: dict):
    is_dict = data.transform(lambda x: x.apply(type).eq(list)).any()
    posible_expandir = list(is_dict[is_dict==True].index)
    try:
        Expandir = [i for i in posible_expandir if diccionario[i]['valores'] is None]
        print(Expandir)
    except Exception as e:
        print("Error en la malla de validación")
        print(e)
    
    for columna in Expandir:
        try:
            result = expand_data_frame(columna, data)
            if len(result.columns) == 1:
                data = data.drop(columns = columna)
            data = pd.concat([data, result], axis = 1)
        except Exception as e:
            print("Problemas con la expansión de la columna {}".format(columna))
            print(e)
    
    return data
    

In [86]:
df = EmprendimientoColectivo.dataframe.copy()

In [87]:
result = expand_cols(df, malla)

['CONOCIMIENTOS_HABILIDADES_TECNICAS', 'CONOCIMINETOS_HABILIDADES_ADMON', 'producto/servicio', 'DireccionTVia', 'Direccion_TManzana']


In [88]:
result['CONOCIMIENTOS_HABILIDADES_TECNICAS']

0                                    [Canales directos ]
1                           [Manipulación de alimentos ]
2                              [Manejo de herramientas ]
3      [Capacitaciones de comercialización y emprendi...
4                                            [Tecnicas ]
                             ...                        
196                         [Finanzas, Diseño e figuras]
197    [Profesionales capacitaciones civil ,familia y...
198    [Soldadura electrica con fundicion con dioxido...
199    [Adquirir conocimientos referentes al servicio...
200    [Produccion audiovisual y maquillaje caracteri...
Name: CONOCIMIENTOS_HABILIDADES_TECNICAS, Length: 201, dtype: object

In [83]:
len(df.columns)

223